In [ ]:
#!pip3 install torchaudio==0.9.0

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Датасет

Подключение необходимых библиотек

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import h5py
import librosa
import os, sys
import time
import pandas as pd 
from collections import namedtuple
from sklearn.model_selection import StratifiedShuffleSplit

Константы на весь проект

In [ ]:
FOLDER_CSV = '/content/drive/MyDrive/DataCsv/' # здесь храним csv
SR = 12000 # семплрейт для аудио в Гц. Частота дискретизации
allowed_exts = set(['mp3', 'wav']) # разрешенный тип файлов
column_names = ['id', 'filepath', 'label'] # названия столбцов в csv

Функция для чтения строк в csv 

In [ ]:
def get_rows_from_folders(folder_dataset, folders):
  rows = [] # инициализируем строки
  # получаем для всех папок суммарный список строк - всех файлов в папках
  # label_idx хранит настроение от 0 до 2
  # folder хранит название папки
  # 0 Sad 1 Happy 2 Aggressive
  # потом папки можно расширить, как и данные внутри папок
  for label_idx, folder in enumerate(folders):
    files = os.listdir(os.path.join(folder_dataset, folder))
    files = [f for f in files if f.split('.')[-1].lower() in allowed_exts]
    for fnm in files:
      file_path = os.path.join(folder_dataset, folder, fnm)
      file_id = fnm.split('.')[0]
      file_label = label_idx
      rows.append([file_id, file_path, file_label]) # делаем строку
  print('Строки скопированы. Длина:{}'.format(len(rows))) # выводим кол-во строк
  # на первом этапе кол-во строк должно быть 300
  # если что-то не так лезь проверяй
  print(rows[0])
  print(rows[-1])
  return rows

Функция для чтения с исправленными id

In [ ]:
def get_rows_from_folders2(folder_dataset, folders):
  rows = [] # инициализируем строки
  # получаем для всех папок суммарный список строк - всех файлов в папках
  # label_idx хранит настроение от 0 до 2
  # folder хранит название папки
  # 0 Sad 1 Happy 2 Aggressive
  # потом папки можно расширить, как и данные внутри папок
  i = 0
  for label_idx, folder in enumerate(folders):
    files = os.listdir(os.path.join(folder_dataset, folder))
    files = [f for f in files if f.split('.')[-1].lower() in allowed_exts]
    for fnm in files:
      file_path = os.path.join(folder_dataset, folder, fnm)
      file_id = i
      i += 1
      file_label = label_idx
      rows.append([file_id, file_path, file_label]) # делаем строку
  print('Строки скопированы. Длина:{}'.format(len(rows))) # выводим кол-во строк
  i = 0
  # на первом этапе кол-во строк должно быть 300
  # если что-то не так лезь проверяй
  print(rows[0])
  print(rows[-1])
  return rows

Функция для записи прочитанных строк в csv

In [ ]:
def write_to_csv(rows, column_names, csv_fnm):
  # rows - строки, которые читаем предыдущей функцией
  # column_names в константах название столбцов
  # csv_fnm - имя csv файла
  df = pd.DataFrame(rows, columns = column_names)
  df.to_csv(os.path.join(FOLDER_CSV, csv_fnm))

Далее указываем путь до папок, где лежат данные

После чего запускаем предыдущие функции по парсу в csv файлов из папок Sad Happy Aggressive

In [ ]:
folder_dataset = '/content/drive/MyDrive/Data/' # путь к папке с датасетом
labels = ['Sad', 'Happy', 'Aggressive'] # разметка для настроений
n_label = len(labels) # генерим 0 sad 1 happy 2 aggressive
folders = [s + '/' for s in labels] # список из путей по настроениям
# пример /content/drive/MyDrive/Data/Sad итд

# получить строки для csv файла
rows_dataset = get_rows_from_folders2(folder_dataset, folders)
# записать в csv
write_to_csv(rows_dataset, column_names, 'dataset2.csv')

Строки скопированы. Длина:300
[0, '/content/drive/MyDrive/Data/Sad/S 1.mp3', 0]
[299, '/content/drive/MyDrive/Data/Aggressive/A 0.mp3', 2]


# Mel Spec


Импорт библиотек

In [1]:
import torch
import torch.nn as nn # слои
import torch.nn.functional as F # функции
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler # датасет
import torchaudio # аудио
import pandas as pd
import os, sys
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import torch.optim as optim

device = 'cuda' if torch.cuda.is_available() else 'cpu'

Класс для создания объекта датасета

In [2]:
class myDataset(Dataset):
  def __init__(self, csv_file):
    self.songs = pd.read_csv(csv_file)

  def __len__(self):
    return len(self.songs)

  def __getitem__(self, idx):
    if torch.is_tensor(idx):
      idx = idx.tolist()
    
    song_path = self.songs.iloc[idx,2]
    song_file, sr = torchaudio.load(str(song_path))
    song_label = self.songs.iloc[idx,3]

    return song_file, song_label

#dataset = myDataset(csv_file='/content/drive/MyDrive/DataCsv/dataset2.csv')
#item = dataset.__getitem__(2)
#print(item[0])
#wav_file, sample_rate = torchaudio.load(str(item[0]))


Класс нейронной сети, создание объекта на GPU

In [3]:
'''
  def pathToSpec(self, song_file):
    wav_file, sample_rate = torchaudio.load(song_file) #ТУТ ПУТЬ К ФАЙЛУ
    ms = torchaudio.transforms.MelSpectrogram(
        sample_rate = 12000,
        n_mels = 96, # ПОМЕНЯЙ НА ЗНАЧЕНИЕ СО СТАТЬИ
    )
    melspec = ms(wav_file)
    return melspec
'''

class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.melspec = torchaudio.transforms.MelSpectrogram( # Мелспека torch.Size([4, 1, 128, 2584])
        sample_rate = 12000,
        n_mels = 128,
        n_fft = 2048,
        hop_length = 512,
        power = 1)
    
    # 4 слоя Conv2d
    self.conv1 = nn.Conv2d( # Conv1 torch.Size([4, 2, 126, 2582])
      kernel_size = (3,3),
      in_channels = 1,
      out_channels = 2
    )
    self.pool = nn.MaxPool2d(2, 2) # pool1 torch.Size([4, 2, 63, 1291])

    '''
    self.conv2 = nn.Conv2d( # conv2 torch.Size([4, 4, 61, 1289])
        kernel_size = (3, 3),
        in_channels = 2,
        out_channels = 4
    )
    self.pool = nn.MaxPool2d(4, 2) # pool2 torch.Size([4, 4, 29, 643])

    self.conv3 = nn.Conv2d( # conv3 torch.Size([4, 8, 27, 641])
        kernel_size = (3, 3),
        in_channels = 4,
        out_channels = 8
    )
    self.pool = nn.MaxPool2d(2, 2) # pool3 torch.Size([4, 8, 13, 320])

    self.conv4 = nn.Conv2d( #conv4 torch.Size([4, 16, 11, 318])
        kernel_size = (3, 3),
        in_channels = 8,
        out_channels = 16
    )'''

    self.fc1 = nn.Linear(2 * 63 * 1291, 90)
    self.fc2 = nn.Linear(90, 30)
    self.fc3 = nn.Linear(30, 3)

  def forward(self, x):
    x = self.melspec(x)
    x = self.pool(F.relu(self.conv1(x)))
    '''x = self.pool(F.relu(self.conv2(x)))
    x = self.pool(F.relu(self.conv3(x)))
    x = self.pool(F.relu(self.conv4(x)))'''
    #x = x.view(-1, 16*11*318)
    x = torch.flatten(x, 1)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    #x = torch.flatten(x, 1) # flatten all dimensions except batch
    #x = F.relu(self.fc1(x))
    #x = F.relu(self.fc2(x))
    #x = self.fc3(x)
    return x

net = Net()
#net.to(device)

Разделение датасета на набор для обучения и проверки.
Запуск обучения и сохранение модели

In [4]:
datasetFull = myDataset(csv_file='/content/drive/MyDrive/DataCsv/dataset2.csv')
batch_size = 4
validation_split = .2
shuffle_dataset = True
random_seed = 42

# Создаём индексы данных для обучения(train) и проверки(validation)
dataset_size = datasetFull.__len__()
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
  np.random.seed(random_seed)
  np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Выбираем элементы случайным образом из заданного списка индексов без замены
train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)

# DataLoaders (для загрузки датасета в батчи)
train_loader = torch.utils.data.DataLoader(datasetFull, batch_size=batch_size, 
                                           sampler=train_sampler)
val_loader = torch.utils.data.DataLoader(datasetFull, batch_size=batch_size,
                                                sampler=val_sampler)

classes = ('Sad', 'Happy', 'Aggressive')

# Определяем Loss функцию и Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

# ЗАПУСК ОБУЧЕНИЯ
running_loss = 0.0
num_epochs = 3
for epoch in range(num_epochs):
  for batch_index, (song_file, song_label) in enumerate(train_loader):
    # получаем вход как лист
    #####song_file, song_label = data

    #song_file = song_file.to(device)
    #song_label = song_label.to(device)
    # параметр градиента
    optimizer.zero_grad()

    # forward + baclward + optimize
    outputs = net(song_file)
    loss = criterion(outputs, song_label)
    loss.backward()
    optimizer.step()

    # print statistics
    running_loss += loss.item()
    if batch_index % 20 == 19:    # print every 2000 mini-batches 2000 == 1999
      print('[%d, %5d] loss: %.3f' %
        (epoch + 1, batch_index + 1, running_loss / 2000))
      running_loss = 0.0

print('Finished Training')

MODELPATH = '/content/drive/MyDrive/TrainedModels/test_net3.pth'
torch.save(net.state_dict(), MODELPATH)


[1,    20] loss: 0.014
[1,    40] loss: 0.011
[1,    60] loss: 0.010
[2,    20] loss: 0.009
[2,    40] loss: 0.008
[2,    60] loss: 0.008
[3,    20] loss: 0.005
[3,    40] loss: 0.006
[3,    60] loss: 0.004
Finished Training


Загрузка сохранённой модели и тестирование

In [5]:
# Загрузка модели
MODELPATH = '/content/drive/MyDrive/TrainedModels/test_net3.pth'
net2 = Net()
net2.load_state_dict(torch.load(MODELPATH))
#net2.to(device)

<All keys matched successfully>

КОРРЕКТНОСТЬ ДЛЯ ВСЕЙ СЕТИ

In [6]:
correct = 0
total = 0
# Градиенты не рассчитываются так как мы не тренируемся
with torch.no_grad():
  for data in val_loader:
    song_file, song_label = data
    # Рассчитываем выход, запуская песню в нейронку
    outputs = net2(song_file)
    # Жанр с максимальной энергией это то что мы выбираем как предсказание
    _, predicted = torch.max(outputs.data, 1)
    total += song_label.size(0)
    correct += (predicted == song_label).sum().item()

print('Accuracy of the network: %d %%' % (100 * correct / total))

Accuracy of the network: 75 %


КОРРЕКТНОСТЬ ПО КАЖДОМУ КЛАССУ

In [7]:
# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in val_loader:
        song_file, song_label = data
        outputs = net2(song_file)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for s_label, prediction in zip(song_label, predictions):
            if s_label == prediction:
                correct_pred[classes[s_label]] += 1
            total_pred[classes[s_label]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print("Accuracy for class {:5s} is: {:.1f} %".format(classname,
                                                   accuracy))

Accuracy for class Sad   is: 90.9 %
Accuracy for class Happy is: 62.5 %
Accuracy for class Aggressive is: 68.2 %


ВОТ ЭТОТ КОД НЕ ЗАПУСКАТЬ И ВСТАВЛЯТЬ ДЛЯ ТЕСТА ВНУТРЬ ПЕРЕБОРА ЭПОХ

In [ ]:
# ВОТ ЭТОТ КОД НЕ ЗАПУСКАТЬ И ВСТАВЛЯТЬ ДЛЯ ТЕСТА ВНУТРЬ ПЕРЕБОРА ЭПОХ
num_epochs = 1
for epoch in range(num_epochs):
  for batch_index, (song_file, song_label) in enumerate(train_loader):
    melspec1 = torchaudio.transforms.MelSpectrogram(
        sample_rate = 12000,
        n_mels = 128,
        n_fft = 2048,
        hop_length = 512,
        power = 1)
    audio = melspec1(song_file) # Мелспека torch.Size([4, 1, 128, 2584])
    print('Мелспека {}'.format(audio.shape))
    conv1 = nn.Conv2d(
      kernel_size = (3,3),
      in_channels = 1,
      out_channels = 2
      )
    x = conv1(audio)
    print('Conv1 {}'.format(x.shape)) # Conv1 torch.Size([4, 2, 126, 2582])

    pool = nn.MaxPool2d(2, 2)
    x = pool(x)
    print('pool1 {}'.format(x.shape)) # pool1 torch.Size([4, 2, 63, 1291])
    
    conv2 = nn.Conv2d(
        kernel_size = (3, 3),
        in_channels = 2,
        out_channels = 4
    )
    x = conv2(x)
    print('conv2 {}'.format(x.shape)) # conv2 torch.Size([4, 4, 61, 1289])
    
    pool = nn.MaxPool2d(4, 2)
    x = pool(x)
    print('pool2 {}'.format(x.shape)) # pool2 torch.Size([4, 4, 29, 643])

    conv3 = nn.Conv2d(
        kernel_size = (3, 3),
        in_channels = 4,
        out_channels = 8
    )
    x = conv3(x)
    print('conv3 {}'.format(x.shape)) # conv3 torch.Size([4, 8, 27, 641])

    pool = nn.MaxPool2d(2, 2)
    x = pool(x)
    print('pool3 {}'.format(x.shape)) # pool3 torch.Size([4, 8, 13, 320])

    conv4 = nn.Conv2d(
        kernel_size = (3, 3),
        in_channels = 8,
        out_channels = 16
    )
    x = conv4(x)
    print('conv4 {}'.format(x.shape)) # conv4 torch.Size([4, 16, 11, 318])

    fc1 = nn.Linear(16*11*318, 120)
    x = fc1(x)
    print('fc1 {}'.format(x.shape)) # 

    fc2 = nn.Linear(120, 60)
    x = fc2(x)
    print('fc2 {}'.format(x.shape)) #

    fc3 = nn.Linear(60, 30)
    x = fc3(x)
    print('fc3 {}'.format(x.shape)) # 

    fc4 = nn.Linear(30, 3)
    x = fc4(x)
    print('fc4 {}'.format(x.shape)) # 

    print('======')


Мелспека torch.Size([4, 1, 128, 2584])
Conv1 torch.Size([4, 2, 126, 2582])
pool1 torch.Size([4, 2, 63, 1291])
conv2 torch.Size([4, 4, 61, 1289])
pool2 torch.Size([4, 4, 29, 643])
conv3 torch.Size([4, 8, 27, 641])
pool3 torch.Size([4, 8, 13, 320])
conv4 torch.Size([4, 16, 11, 318])


RuntimeError: ignored